In [36]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import dateutil.parser as parser
from datetime import datetime, date, timedelta
import torch
import skorch
import scipy
import torch.nn as nn
import torch
import torch.nn.functional as F
import sys
from skorch.helper import DataFrameTransformer
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, r2_score
from sklearn.linear_model import SGDClassifier
from skorch import NeuralNetRegressor
from sklearn.pipeline import Pipeline
from sklearn import metrics

In [2]:
merged = pd.read_csv('merged.csv')

all_tweets = merged["text"]
all_tweets = all_tweets.to_list()
for tweet in all_tweets:
    # Remove all the special characters
    processed_tweet = re.sub(r'\W', ' ', tweet)
 
    # remove all single characters
    processed_tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_tweet)
 
    # Remove single characters from the start
    processed_tweet = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_tweet) 
 
    # Substituting multiple spaces with single space
    processed_tweet= re.sub(r'\s+', ' ', processed_tweet, flags=re.I)
 
    # Removing prefixed 'b'
    processed_tweet = re.sub(r'^b\s+', '', processed_tweet)
    
    #removing common pronouns and prepositions
    processed_tweet = re.sub(r'of|to|https|keep|128', '', processed_tweet)
 
    # Converting to Lowercase
    processed_tweet = processed_tweet.lower()
    
    tweet = processed_tweet
    
tfidfv = TfidfVectorizer(max_features=2000)
df2 = tfidfv.fit_transform(all_tweets)
df2array = df2.toarray()

In [55]:
y = merged['Change']
print(y.max())
print(y.min())
print(y.mean())
bins = np.arange(start=-4.5, stop=4.5, step = 0.5)
# bins = np.array([-1,1])
print(bins)
ybin = np.digitize(y, bins)
print(ybin)

4.04
-4.57
-0.12666603819921832
[-4.5 -4.  -3.5 -3.  -2.5 -2.  -1.5 -1.  -0.5  0.   0.5  1.   1.5  2.
  2.5  3.   3.5  4. ]
[14 14 14 ... 11 11 11]


In [56]:
#Selecting the data and splitting into train and test
y = merged['Change']
print(type(y))
# X = df3
# X = df2
# X = df2array
X = all_tweets
from sklearn.model_selection import train_test_split  
x_train, x_test, y_train, y_test = train_test_split(X, ybin, test_size=0.2, random_state=0)

# train_holdout_data.to_csv('train_holdout_data.csv', index=False)
# x_holdout.to_csv('x_holdout.csv', index=False)
# y_holdout.to_csv('y_holdout.csv', index=False)
# train_data.to_csv('train_data.csv', index=False)
# x_train.to_csv('x_train.csv', index=False)
# y_train.to_csv('y_train.csv', index=False)
# # x_test.to_csv('x_test.csv', index=False)
# y_test.to_csv('y_test.csv', index=False)

# # train_holdout_data = pd.read_csv('train_holdout_data.csv')
# # x_holdout = pd.read_csv('x_holdout.csv')
# # y_holdout = pd.read_csv('y_holdout.csv')
# # x_train = pd.read_csv('x_train.csv')
# y_train = pd.read_csv('y_train.csv')
# # x_test = pd.read_csv('x_test.csv')
# y_test = pd.read_csv('y_test.csv')

<class 'pandas.core.series.Series'>


In [33]:
print(x_train)
print(y_train)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



AttributeError: 'list' object has no attribute 'shape'

In [57]:
pipe = Pipeline([("tfidf_vector_com", TfidfVectorizer(
               input="array",
               norm="l2",
               max_features=None,
               sublinear_tf=True,
               stop_words="english")),
          ("clf", SGDClassifier(
               loss="log",
               penalty="l2",
               class_weight="balanced",
               tol=0.001))])

pipe.fit(x_train, y_train)

pred_test = pipe.predict(x_test)
pred_train = pipe.predict(x_train)
print("test accuracy", str(np.mean(pred_test == y_test)))
print(metrics.classification_report(y_test, pred_test))

test accuracy 0.24928693667997717
              precision    recall  f1-score   support

           0       0.20      0.31      0.25      3164
           2       0.07      0.34      0.12      1339
           3       0.18      0.31      0.23      3000
           4       0.32      0.15      0.21      8315
           5       0.26      0.33      0.29      5130
           6       0.39      0.42      0.41      5965
           7       0.26      0.14      0.18      6684
           8       0.34      0.23      0.27      9299
           9       0.24      0.36      0.29     14536
          10       0.26      0.22      0.24      3858
          11       0.26      0.22      0.24     11621
          12       0.24      0.14      0.18      7409
          13       0.25      0.30      0.27      2849
          14       0.35      0.15      0.21      9095
          15       0.20      0.30      0.24      3089
          16       0.21      0.25      0.23      4407
          18       0.21      0.28      0.24    